In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os,errno
import sys
import datetime as dt
import pandas as pd
import glob

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
month_days=[31,28,31,30,31,30,31,31,30,31,30,31]
yrs=np.arange(2000,2022,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    count=0
    month_days[1]=28
    if year%4==0:
        month_days[1]=29
    start=0
    end=12
    if year==2000:
        start=5
    if year==2021:
        end=8
    for j in range(start,end):
        print(j+1)
        if j<9:
            month_str='0'+str(j+1)
        else:
            month_str=str(j+1)
        for k in range(month_days[j]):
            if k<9:
                day_str='0'+str(k+1)
            else:
                day_str=str(k+1)
            
            files=glob.glob(dir_data+'GPCPDAY_L3_'+str(year)+month_str+day_str+'*')
            
            if len(files)==1:
                ds=xr.open_dataset(files[0])
                precip=ds['precip']
                
                if count==0:
                    precip_concat=precip
                    count=1
                else:
                    precip_concat=xr.concat([precip_concat,precip],dim='time')
                
            else:
                print('ERROR')
                print(str(year)+month_str+day_str)
    
    dk=xr.Dataset()
    dk['precip']=(('time','lat','lon'),precip_concat.values)
    dk.coords['time']=precip_concat.time
    dk.coords['lat']=precip_concat.lat.values
    dk.coords['lon']=precip_concat.lon.values
    dk['precip'].attrs["units"]='mm*day-1'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    try:
        os.remove(dir_data+'gpcp3_precip_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'gpcp3_precip_'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'gpcp3_precip_'+str(2021)+'.nc')
test=ds['precip'][100,:,:]
print(ds)

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=50,cmap=plt.cm.RdYlGn_r)
ax.coastlines(resolution='10m')
plt.show()